# Projeto 1 - Ciência dos Dados

Nome: Danilo Ramos

Nome: Leonardo Veras

Nome: Thomas Ganut

Atenção: Serão permitidos grupos de até 4 pessoas, mas com uma rubrica mais exigente. 

___
Carregando algumas bibliotecas:

In [113]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import display
import re

In [114]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\leovm\OneDrive - Insper - Institudo de Ensino e Pesquisa\Insper\Ciencia dos Dados\Projeto 1\C-Dados-Porreta


Carregando a base de dados com as mensagens dos seus arquivos:

In [115]:
arquivo_treino = 'dados_treino_QUARTETO_Danilo_Leonardo_Thomas.csv'
train = pd.read_csv(arquivo_treino)

train.head(5)

,Comment,Topic
0,If you pile up electrons in a hypothetical cha...,Physics
1,Is it the infamous kill-you-dead Brazilian wan...,Biology
2,What a dumb fucking question. No you moron the...,Biology
3,Yes! Humans in North America have really led ...,Biology
4,Take my award. What an underrated comment.,Chemistry


In [116]:
test = pd.read_csv('dados_teste_QUARTETO_Danilo_Leonardo_Thomas.csv')
test.head(5)

,Comment,Topic
0,Wait wouldn’t the wings rip off before the res...,Biology
1,It's pretty much the same idea behind people a...,Biology
2,"Yeah, good to know. We had a crazy infestation...",Biology
3,It’s a kiwi doggy,Biology
4,You're absolutely right about the flu not kill...,Biology


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o contexto referente aos rótulos cujas mensagens (ou reviews) deverão ser classificadas.

Será desenvolvido um sistema automático, utilizando o Teorema de Bayes, para classificar textos científicos em física, química e biologia. Uma prova de conceito será implementada para treinar e testar o classificador, garantindo a separação clara dos textos nas respectivas áreas.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [117]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[´"!-.:?;$'')(&€™/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [118]:
for i, frase in enumerate(train.Comment):
    train.Comment[i] = cleanup(frase.lower())

In [119]:
train_physics = train.loc[train.Topic == 'Physics']
train_biology = train.loc[train.Topic == 'Biology']
train_chemistry = train.loc[train.Topic == 'Chemistry']

In [120]:
list_chemistry: list = []
for frase in train_chemistry.Comment:
    frase = frase.split()
    for word in frase:
        list_chemistry.append(word)

serie_chemistry = pd.Series(list_chemistry)


list_physics: list = []
for frase in train_physics.Comment:
    frase = frase.split()
    for word in frase:
        list_physics.append(word)

serie_physics = pd.Series(list_physics)


list_biology: list = []
for frase in train_biology.Comment:
    frase = frase.split()
    for word in frase:
        list_biology.append(word)

serie_biology = pd.Series(list_biology)

In [121]:
list_train: list = []
for frase in train.Comment:
    frase = frase.split()
    for word in frase:
        list_train.append(word)
        
serie_train = pd.Series(list_train)

In [122]:
P_C = len(serie_chemistry)/len(serie_train)
P_B = len(serie_biology)/len(serie_train)
P_P = len(serie_physics)/len(serie_train)
print(P_C, P_B, P_P)
print(P_C + P_B + P_P)


0.3012356545583805 0.3779207149869664 0.3208436304546532
1.0


In [123]:
# #Treinamento
tabela_c_relativa = serie_chemistry.value_counts(True)
tabela_p_relativa = serie_physics.value_counts(True)
tabela_b_relativa = serie_biology.value_counts(True)
print(tabela_p_relativa)

tabela = serie_train.value_counts()

# Physics
tabela_P_raw = serie_physics.value_counts()
tabela_P_freq = tabela_P_raw.sum()
tabela_P_voc = tabela_P_raw.count()

# Chemistry
tabela_C_raw = serie_chemistry.value_counts()
tabela_C_freq = tabela_C_raw.sum()
tabela_C_voc = tabela_C_raw.count()

# Biology
tabela_B_raw = serie_biology.value_counts()
tabela_B_freq = tabela_B_raw.sum()
tabela_B_voc = tabela_B_raw.count()

stop_words = tabela.index[:15].tolist()
#print(stop_words)

def cleanup_stopwords(frase):
    punctuation = '[´"!-.:?;$'')(&€™/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', frase)
    text_subbed = text_subbed.lower()
    for palavra in stop_words:
        text_subbed = text_subbed.replace((f' {palavra} '), ' ')

    return text_subbed
frase_teste = 'My name is the Leo, and I play a lot of Minecraft!'
#print(cleanup_stopwords(frase_teste))

the               0.053559
to                0.026505
a                 0.024711
of                0.022643
is                0.021040
                    ...   
gaps              0.000021
commenting        0.000021
pressurevacuum    0.000021
blocking          0.000021
duckstream        0.000021
Name: proportion, Length: 6760, dtype: float64


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [124]:
for i, frase in enumerate(test.Comment):
    test.Comment[i] = cleanup(frase.lower())


def algoritmo_1(frase: str):
    probFraseDadoP = 1
    probFraseDadoC = 1
    probFraseDadoB = 1
    

    for palavra in frase.split():
        if palavra in serie_train:
            probFraseDadoP *= tabela_p_relativa[palavra]
            probFraseDadoC *= tabela_c_relativa[palavra]
            probFraseDadoB *= tabela_b_relativa[palavra]
        else:
            probFraseDadoP *= 0.00001
            probFraseDadoC *= 0.00001
            probFraseDadoB *= 0.00001
        
    prob_P_dado_frase = probFraseDadoP * P_P

    prob_C_dado_frase = probFraseDadoC * P_C
    prob_B_dado_frase = probFraseDadoB * P_B

    
    if  prob_P_dado_frase > prob_C_dado_frase:
        if prob_P_dado_frase > prob_B_dado_frase:
            return 'Physics'
        else:
            return 'Biology'
    else:
        if prob_C_dado_frase > prob_B_dado_frase:
            return 'Chemistry'
        else:
            return 'Biology'
        


#frases_physics = test.loc[test.Topic == 'Physics']
#frases_chemistry = test.loc[test.Topic == 'Chemistry']
#frases_biology = test.loc[test.Topic == 'Biology']



In [125]:
def algoritmo_laplace(frase: str):
    alpha = 1 #Suavizador de Laplace
    probFraseDadoP = 1
    probFraseDadoC = 1
    probFraseDadoB = 1

    for palavra in frase.split():
        if palavra in serie_train:
            probFraseDadoP *= (tabela_P_raw[palavra] + alpha) / (tabela_P_freq + alpha * tabela_P_voc)
            probFraseDadoC *= (tabela_C_raw[palavra] + alpha) / (tabela_C_freq + alpha * tabela_C_voc)
            probFraseDadoB *= (tabela_B_raw[palavra] + alpha) / (tabela_B_freq + alpha * tabela_B_voc)
        else:
            probFraseDadoP *= alpha / (tabela_P_freq + alpha * tabela_P_voc)
            probFraseDadoC *= alpha / (tabela_C_freq + alpha * tabela_C_voc)
            probFraseDadoB *= alpha / (tabela_B_freq + alpha * tabela_B_voc)
        
    prob_P_dado_frase = probFraseDadoP * P_P
    prob_C_dado_frase = probFraseDadoC * P_C
    prob_B_dado_frase = probFraseDadoB * P_B


    if  prob_P_dado_frase > prob_C_dado_frase:
        if prob_P_dado_frase > prob_B_dado_frase:
            return 'Physics'
        else:
            return 'Biology'
    else:
        if prob_C_dado_frase > prob_B_dado_frase:
            return 'Chemistry'
        else:
            return 'Biology'

In [126]:
frase_teste = cleanup_stopwords('For sure, were still pushing the frontiers our understanding. And no being in the field of quantum optics, Ill admit my statement falls short here. However, Ive yet to see any "magic" from quantum optics abused to make a marvel movie more "sciencey" for people. Meanwhile, well-understood phenomena that are easy to think about for waves continues to be magical because master must be little balls.'.lower())
#print(frase_teste)
print('Sem Laplace: \n', algoritmo_1(frase_teste), '\n')
print('Com Laplace: \n', algoritmo_laplace(frase_teste))

Sem Laplace: 
 Biology 

Com Laplace: 
 Chemistry


In [131]:
validacao = train
validacao['algo1_certa'] = 'NaN'
validacao['algoL_certa'] = 'NaN'
validacao['algo1'] = 'NaN'
validacao['algoL'] = 'NaN'

for i, frase in enumerate(test.Comment):
    frase = cleanup_stopwords(frase)
    validacao['algo1'][i] = algoritmo_1(frase)
    validacao['algoL'][i] = algoritmo_laplace(frase)
  
    if validacao['algo1'][i] == validacao['Topic'][i]:
        validacao['algo1_certa'][i] = True
    else:
        validacao['algo1_certa'][i] = False

    if validacao['algoL'][i] == validacao['Topic'][i]:
        validacao['algoL_certa'][i] = True
    else:
        validacao['algoL_certa'][i] = False

validacao

,Comment,Topic,algo1_certa,algoL_certa,algo1,algoL
0,if you pile up electrons in a hypothetical cha...,Physics,False,False,Biology,Chemistry
1,is it the infamous killyoudead brazilian wande...,Biology,True,False,Biology,Chemistry
2,what a dumb fucking question no you moron ther...,Biology,True,False,Biology,Chemistry
3,yes humans in north america have really led t...,Biology,True,False,Biology,Chemistry
4,take my award what an underrated comment,Chemistry,False,True,Biology,Chemistry
...,...,...,...,...,...,...
4195,i don’t know but the sooner you can get it to ...,Biology,NaN,NaN,NaN,NaN
4196,this man is a troll though the real culprit ar...,Biology,NaN,NaN,NaN,NaN
4197,there needs to be some automod that removes th...,Biology,NaN,NaN,NaN,NaN
4198,yeah this should be the official name a ducks...,Physics,NaN,NaN,NaN,NaN


In [132]:
certos_algo1 = validacao.algo1_certa.value_counts(True)
certos_algoL = validacao.algoL_certa.value_counts(True)

print('Acertos sem Laplace: ')
print(f'{certos_algo1[True].round(2)*100}%')
print('\nAcertos com Laplace:')
print(f'{certos_algoL[True].round(2)*100}%')

Acertos sem Laplace: 
17.0%

Acertos com Laplace:
15.0%


In [135]:
# pd.crosstab(test.Topic, test.algo1, normalize=True)
print(test.algo1.value_counts())
print('\n')
print(test.algoL.value_counts())

algo1
Biology    1800
Name: count, dtype: int64


algoL
Chemistry    1564
Biology       165
Physics        71
Name: count, dtype: int64


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Leia atentamente a rubrica colocada no enunciado do Projeto 1 (última página). <br>
Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas mensagens, mas tendem a melhorar na classificação das mensagens. Ex: stemming, lemmatization, stopwords.
* CONSIDEROU arquivo com três categorias na classificação das variáveis (OBRIGATÓRIO PARA QUARTETOS, sem contar como item avançado)
* CONSTRUIU o cálculo das probabilidades corretamente utilizando bigramas E apresentou referência sobre o método utilizado.
* EXPLICOU porquê não pode usar novas mensagens classificadas pelo próprio classificador como amostra de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários diferentes, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* REFLETE criticamente sobre os resultados obtidos, identificando limitações do modelo e sugerindo possíveis melhorias ou diferentes abordagens com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa).
* DOCUMENTOU bem o código, com explicações claras para cada etapa do processo, incluindo o raciocínio por trás das decisões de modelagem e das transformações de dados.


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**